In [15]:
import json
import pandas as pd
import requests

In [16]:
follows = pd.read_csv('../shared-folder-gald/data/follow-link.csv')

In [17]:
usernames = pd.unique(follows[['source', 'target']].values.ravel()).tolist()

In [18]:
with open("../shared-folder-gald/keys2.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [19]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [20]:
# def get_access_token(client_key, client_secret):
#     endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {
#         'client_key': client_key,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials',
#     }

#     response = requests.post(endpoint_url, headers=headers, data=data)
#     response_json = response.json()

#     if response.status_code == 200:
#         print(response_json)  # Debugging print
#         return response_json.get('access_token', 'No access token found')
#     else:
#         print(f"Error {response.status_code}: {response_json}")  # Debugging print
#         return response_json

In [21]:
def get_repost_info(username, access_token,fields ="id, username, favorites_count" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/reposted_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [22]:
access_token = get_access_token(client_key, client_secret)

keeping track with calls
27.02 - we ran keys2 unril 1000 

1.03 - keys2 1001:2002

1.03 - keys1 2002:3003

2.03 - keys1 3003:4004

2.03 - keys2 4004:5005

3.03 - keys1 5005:6006

3.03 - keys2 6006:7007

4.03 - keys1 7007:8008

4.03 - keys2 8008:9009


In [23]:
reposted_videos = []
for username in usernames[8008:9009]:
    reposted_videos.append(get_repost_info(username, access_token))

In [24]:
reposted_videos

[{'cursor': -1,
  'has_more': False,
  'reposted_videos': [{'favorites_count': 1131,
    'id': 7442332737937181998,
    'username': 'keepemhonest1'},
   {'favorites_count': 420,
    'id': 7401461905929276705,
    'username': 'boxeboxingbr'}]},
 {'cursor': -1, 'has_more': False},
 {'cursor': 21,
  'has_more': True,
  'reposted_videos': [{'favorites_count': 1,
    'id': 7464961199885520150,
    'username': 'rock.nd.rose'},
   {'favorites_count': 1,
    'id': 7461359278058753312,
    'username': 'abiismstore'},
   {'favorites_count': 9006,
    'id': 7470339676801486086,
    'username': 'fredighoul'},
   {'id': 7477200510290808086,
    'username': 'userrr1729291',
    'favorites_count': 802},
   {'favorites_count': 354,
    'id': 7472394801720184086,
    'username': 'montoya_1903'},
   {'favorites_count': 2, 'id': 7467890857455258902, 'username': 'tximista6'},
   {'favorites_count': 4, 'id': 7459739817534901537, 'username': 'elenamesag'},
   {'favorites_count': 1,
    'id': 746232509317631

In [25]:
#saving global reposts for weights in the network
import csv
with open('reposted_videos_raw', 'w') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(reposted_videos)

In [26]:
combined = {}
for i, username in enumerate(usernames[8008:9009]):
    combined[username] = reposted_videos[i]
    
combined 


{'sparkflames.com': {'cursor': -1,
  'has_more': False,
  'reposted_videos': [{'favorites_count': 1131,
    'id': 7442332737937181998,
    'username': 'keepemhonest1'},
   {'favorites_count': 420,
    'id': 7401461905929276705,
    'username': 'boxeboxingbr'}]},
 'solartiara': {'cursor': -1, 'has_more': False},
 'lorenzavintage': {'cursor': 21,
  'has_more': True,
  'reposted_videos': [{'favorites_count': 1,
    'id': 7464961199885520150,
    'username': 'rock.nd.rose'},
   {'favorites_count': 1,
    'id': 7461359278058753312,
    'username': 'abiismstore'},
   {'favorites_count': 9006,
    'id': 7470339676801486086,
    'username': 'fredighoul'},
   {'id': 7477200510290808086,
    'username': 'userrr1729291',
    'favorites_count': 802},
   {'favorites_count': 354,
    'id': 7472394801720184086,
    'username': 'montoya_1903'},
   {'favorites_count': 2, 'id': 7467890857455258902, 'username': 'tximista6'},
   {'favorites_count': 4, 'id': 7459739817534901537, 'username': 'elenamesag'},


In [27]:
rows = []

for key, value in combined.items():
    if value and 'reposted_videos' in value:  # Ensure key has data
        for video in value['reposted_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'video_id': video['id'],  # Video ID
                'creator': video['username'], # Video username
                'favourites': video["favorites_count"]
            })

# Create DataFrame
reposted_df = pd.DataFrame(rows)

# Display DataFrame
print(reposted_df)

             username             video_id                   creator  \
0     sparkflames.com  7442332737937181998             keepemhonest1   
1     sparkflames.com  7401461905929276705              boxeboxingbr   
2      lorenzavintage  7464961199885520150              rock.nd.rose   
3      lorenzavintage  7461359278058753312               abiismstore   
4      lorenzavintage  7470339676801486086                fredighoul   
...               ...                  ...                       ...   
7378     sovaportrait  7301447125764721925                urgentseas   
7379     sovaportrait  7361467942418124075  shopthebutterflyeffect13   
7380     sovaportrait  7266369723904904450             jasmineday.33   
7381     sovaportrait  7253596892641103147               coralpeters   
7382     sovaportrait  7246280050436345134             ilovepet55555   

      favourites  
0           1131  
1            420  
2              1  
3              1  
4           9006  
...          ...  
73

In [28]:
reposted_df.to_csv("reposted_videos.csv", mode="a", index=False, header=False)